In [8]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import json
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, Activation, Embedding
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import json
from keras.models import model_from_json

# Import libraries
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
# nltk.download()
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

import string
import re
from keras.preprocessing.text import Tokenizer

In [20]:
# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=10000)
# for human-friendly printing
labels = ['fake', 'real']

# read in our saved dictionary
with open('./Dictionary_Models/keras_dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [23]:
json_file = open('./NN_Models/blstm_keras_embed_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)

model.load_weights('./NN_Models/blstm_keras_embed_model.h5')


In [ ]:
while 1:
    #input a text string
    text = input("Enter a tweet to be evaluated: ")
    
    # check if it is valid string
    if len(text) == 0 or text == 'quit':
        break
        
    tweet = text
    
    nltk.download('stopwords')
    nltk.download('punkt')
    #punctutation removal
    def clean_text(text):
        '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
        and remove words containing numbers.'''
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text
    
    # Tockenization
    def tokenization(text):
        text = re.split('\W+', text)
        return text

    # stopword removal
    stopword = nltk.corpus.stopwords.words('english')

    def remove_stopwords(text):
        text = [word for word in text if word not in stopword]
        return text

    # return to string
    def listToString(s):     
        # initialize an empty string 
        str1 = " " 
        # return string   
        return (str1.join(s))
    
    from nltk.tokenize import sent_tokenize, word_tokenize
    from nltk.stem import PorterStemmer

    porter =  PorterStemmer()

    def sentenceStemmer(text):
        words = word_tokenize(text)
        corpus = []
        for word in words:
            corpus.append(porter.stem(word))
            corpus.append(" ")
        #end for
        return "".join(corpus)
    #end sentenceStemmer
    
    def convert_text_to_index_array(text):
        words = kpt.text_to_word_sequence(text)
        # words = pad_sequences(words, padding='post', maxlen=23)
        # print(text)
        wordIndices = []
        for word in words:
            if word in dictionary:
                wordIndices.append(dictionary[word])
        return wordIndices
    
    # Clean the text
    text = clean_text(text)
    text = tokenization(text)
    text = remove_stopwords(text)
    text = listToString(text)
    text = sentenceStemmer(text)
    
    # evaluating the text
    text = convert_text_to_index_array(text)

    padded_text = pad_sequences([text], padding='post', maxlen=100)
    
    pred = model.predict_generator(padded_text)

    def rounding(results):
        '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
        if results < 0.5:
            return 0
        else:
            return 1

    predictions_final = [rounding(x) for x in pred]
    
    if predictions_final[0] == 0:
        print("\nFake tweet\n")
    if predictions_final[0] == 1:
        print("\nReal tweet\n")

#end while    

Enter a tweet to be evaluated: Allah please safe lest fire consume all of us


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/patrick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/patrick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Real tweet

Enter a tweet to be evaluated: Fire on the mountain coming for all of us because of our deeds!


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/patrick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/patrick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Fake tweet



# Tweepy Connection